In [1]:
import tensorflow as tf

NUM_CLASSES = 10
IMG_SIZE = (224, 224)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

model_crop = tf.keras.Sequential([
    tf.keras.layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input), # Wrapped the preprocess_input function in a Lambda layer
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

model_crop.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_crop.summary()


ImportError: cannot import name 'runtime_version' from 'google.protobuf' (c:\Users\netvo\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\__init__.py)

In [ ]:
import os
import shutil
import random

SOURCE_DIR = "model_1_data"          # your current dataset
DEST_DIR = "model_1_dataset"      # new folder to be created

TRAIN_RATIO = 0.7
VALID_RATIO = 0.2
TEST_RATIO  = 0.1

# Create base directories
for split in ["train", "valid", "test"]:
    os.makedirs(os.path.join(DEST_DIR, split), exist_ok=True)

# Loop through each class folder
for class_name in os.listdir(SOURCE_DIR):
    class_path = os.path.join(SOURCE_DIR, class_name)

    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    total = len(images)
    train_end = int(total * TRAIN_RATIO)
    valid_end = train_end + int(total * VALID_RATIO)

    splits = {
        "train": images[:train_end],
        "valid": images[train_end:valid_end],
        "test": images[valid_end:]
    }

    for split, split_images in splits.items():
        split_class_dir = os.path.join(DEST_DIR, split, class_name)
        os.makedirs(split_class_dir, exist_ok=True)

        for img in split_images:
            src = os.path.join(class_path, img)
            dst = os.path.join(split_class_dir, img)
            shutil.copy(src, dst)

    print(f"✔ Split completed for class: {class_name}")

print("\n✅ Dataset successfully split into train / valid / test")


✔ Split completed for class: Cotton
✔ Split completed for class: Maize
✔ Split completed for class: Pigeon Pea
✔ Split completed for class: Soybean
✔ Split completed for class: Suagrcane

✅ Dataset successfully split into train / valid / test


In [ ]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    "model_1_dataset/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

valid_ds = tf.keras.utils.image_dataset_from_directory(
    "model_1_dataset/valid",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "model_1_dataset/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    shuffle=False
)
#this part is used to separate datasets into train valid and test 

class_names = train_ds.class_names
num_classes = len(class_names)

print(class_names)
print(num_classes)
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

AUTOTUNE = tf.data.AUTOTUNE

def preprocess(ds):
    return ds.map(
        lambda x, y: (preprocess_input(x), y),
        num_parallel_calls=AUTOTUNE
    ).prefetch(AUTOTUNE)

train_ds = preprocess(train_ds)
valid_ds = preprocess(valid_ds)
test_ds  = preprocess(test_ds)


from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


import json

with open("crop_class_names.json", "w") as f:
    json.dump(class_names, f)

model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=5
)

model.save("new_model.keras") 


c:\Users\netvo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Found 1929 files belonging to 5 classes.
Found 723 files belonging to 5 classes.
Found 386 files belonging to 5 classes.
['Cotton', 'Maize', 'Pigeon Pea', 'Soybean', 'Suagrcane']
5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,587,205 (9.87 MB)

 Trainable params: 329,221 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 55s 818ms/step - accuracy: 0.9606 - loss: 0.1067 - val_accuracy: 0.9959 - val_loss: 0.0096
Epoch 2/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 45s 742ms/step - accuracy: 0.9984 - loss: 0.0040 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 3/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 46s 749ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.9986 - val_loss: 0.0021
Epoch 4/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 46s 753ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.9986 - val_loss: 0.0034
Epoch 5/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 46s 750ms/step - accuracy: 1.0000 - loss: 3.6029e-04 - val_accuracy: 0.9986 - val_loss: 0.0013


In [2]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load model & classes
model = tf.keras.models.load_model("crop_identification_model.keras")

with open("crop_class_names.json") as f:
    class_names = json.load(f)

IMG_SIZE = (224, 224)

def predict_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    preds = model.predict(img_array)[0]

    confidence = np.max(preds) * 100
    index = np.argmax(preds)
    label = class_names[index]

    # Entropy check
    entropy = -np.sum(preds * np.log(preds + 1e-10))

    if label == "Unknown" or confidence < 70 or entropy > 1.2:
        return "Unknown Crop", confidence

    return label, confidence


crop, conf = predict_image("C:/projects/Plant_Disease_Detection/model_1_data/Suagrcane/image001.jpg")
print(f"Prediction: {crop} ({conf:.2f}%)")


ImportError: cannot import name 'runtime_version' from 'google.protobuf' (c:\Users\netvo\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\__init__.py)